In [5]:
using Makie
using WGLMakie
using LinearAlgebra

In [6]:
function get_H(Nm)
    ωM = 1000
    ωc = 1000
    LS = 1e-12 / 5.29177210903e-11 # meters -> bohr
    ϵ = 1/(4π) # Permitivity in a.u
    
    # hbar is just a conversion factor here (cm-1 -> hartree)
    ħ = 1/219474.6
    
    μ = 10 * ħ
    
    # Initiate H
    H = zeros(ComplexF64, Nm+1, Nm+1)
    
    # Get photonic diagonal terms
    for i = 1:Nm
        # Each i represents a state where molecule i is excited, thus there are 0 photons left
        H[i+1,i+1] = ħ*ωM
    end
    
    # State with all molecules in the ground state and a photon
    H[1, 1] = ħ*ωc
    
    # Get off diagonal terms
    for i = 1:Nm
        
        H[1, i+1] =  im * √(ħ*ωc/ 2*ϵ*LS)*μ
        H[i+1, 1] = -im * √(ħ*ωc/ 2*ϵ*LS)*μ
        
    end
    
    return H
    
end

get_H (generic function with 1 method)

In [7]:
function splitting_plot(Nmax, step=1)
    ωM = 1000
    ωc = 1000
    LS = 1e-12 / 5.29177210903e-11 # meters -> bohr
    ϵ = 1/(4π) # Permitivity in a.u
    
    # hbar is just a conversion factor here (cm-1 -> hartree)
    ħ = 1/219474.6
    
    μ = 100 * ħ
    
    Ω = √(ħ*ωc/ 2*ϵ*LS)* (μ / ħ)
    
    Nvals = collect(1:step:Nmax)
    Δvals = zeros(length(Nvals))
    
    for i = 1:length(Nvals)
        N = Nvals[i]
        H = get_H(N)
        scale = H[1,1]
        H = H ./ scale # Get a matrix with more stable numbers
        e,c = eigen(H)
        Δvals[i] = (e[end] - e[1])* 219474.6 * scale
    end
    fig = Figure()
    ax = Axis(fig[1,1])
    scatter!(Nvals, Δvals)
    lines!(1:Nmax, [2*Ω*√n for n = 1:Nmax], color=:red)
    display(fig)
end

splitting_plot (generic function with 2 methods)

In [8]:
splitting_plot(500, 20)

In [15]:
H = get_H(10)
e,C = eigen(H)
C[1,:]

11-element Vector{ComplexF64}:
 0.0 - 0.7071067811855126im
 0.0 + 0.0im
 0.0 + 2.7130311885947833e-12im
 0.0 - 3.0735154005201437e-12im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 0.0im
 0.0 + 1.1287082379851654e-12im
 0.0 + 0.0im
 0.0 - 2.1827852025868566e-12im
 0.0 - 0.7071067811875825im

In [100]:
adjoint(C)*H*C |> diag

4-element Vector{ComplexF64}:
  0.004554875194928473 + 0.0im
 0.0045563359040180496 + 0.0im
 0.0045563359040180496 + 0.0im
 0.0045577966131076265 + 0.0im

In [96]:
g = adjoint(C)[:,1]

1.0 + 0.0im

In [95]:
abs(dot(C[:,1], g))^2

0.3333333333333333

In [9]:
function propagate(state, evals, tf, tstep)
    tvals = 0:tstep:tf
    gvals = zeros(length(tvals))
    for i = 1:length(tvals)
        t = tvals[i]
        prop = diagm([exp(-im*E*t) for E in evals])
        new = prop * state
        gvals[i] = abs(dot(state, new))^2
    end
    
    lowest,idx = findmin(gvals)

    println("Minimim probability ($lowest) at t = $(tvals[idx])")
    lines(tvals, gvals)
end

propagate (generic function with 1 method)

In [12]:
H = get_H(1) 

e,C = eigen(H)

#e 

g = [C[1,1], C[1,2]]
#g = adjoint(C)[:,2]
#g = transpose(C)[:,2]

propagate(g, e, 100000000, 100)

Minimim probability (2.563462939541844e-13) at t = 93129400


In [11]:
lines(rand(3), rand(3))